In [11]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn
import pathlib as pl

In [12]:
dictionarypath = '../dataset/dictionary/Loughran-McDonald_MasterDictionary_1993-2021.csv'
wds_dict = pd.read_csv(dictionarypath)
positive_wds = [row['Word'].lower() for i, row in wds_dict.iterrows() if row['Positive']!=0]
negative_wds = [row['Word'].lower() for i, row in wds_dict.iterrows() if row['Negative']!=0]

In [13]:
path = '../dataset/finRep/FinancialPhraseBank/all-data.csv'
df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])
# print(df[:10])

In [14]:
# to install keybert
# !pip install nltk
# !pip install keybert
# !pip install sentence-transformers
# !pip install git+https://github.com/LIAAD/yake
### To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [42]:
def compare_words(words):
    result = 0
    for word1 in words:
        word1 = word1.split()
        for word in word1:
            if word in positive_wds:
                result = result + 1
                print ("(%s,%s)"%(word,"positive"))
            elif word in negative_wds:
                result = result - 1
                print ("(%s,%s)"%(word,"negative"))
            else:
                print ("(%s,%s)"%(word,"neutral"))

In [44]:
# choose a random sample
text = df.sample(1)['sentence'].iloc[0]
label = df.sample(1)['sentiment'].iloc[0]
print(text)
print("the sentiment is : %s"%label)

Talvivaara Mining Company Plc Talvivaara Mining Company is an internationally significant base metals producer with its primary focus on nickel and zinc using a technology known as bioheapleaching to extract metals out of ore .
the sentiment is : positive


Rake

In [45]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['talvivaara mining company plc talvivaara mining company', 'internationally significant base metals producer', 'extract metals', 'zinc using', 'technology known', 'primary focus', 'ore', 'nickel', 'bioheapleaching']


In [46]:
compare_words(keyword_extracted)

    

(talvivaara,neutral)
(mining,neutral)
(company,neutral)
(plc,neutral)
(talvivaara,neutral)
(mining,neutral)
(company,neutral)
(internationally,neutral)
(significant,neutral)
(base,neutral)
(metals,neutral)
(producer,neutral)
(extract,neutral)
(metals,neutral)
(zinc,neutral)
(using,neutral)
(technology,neutral)
(known,neutral)
(primary,neutral)
(focus,neutral)
(ore,neutral)
(nickel,neutral)
(bioheapleaching,neutral)


In [ ]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [ ]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [ ]:
print(f'Text: {text}.\nLength: {len(text.split())}')
best_terms = extract_terms(text)

for terms_and_score in best_terms:
  print(terms_and_score)

Text: The Marubeni Group focuses on creating `` value chain '' from upstream to downstream , encompassing a wide variety of business fields , including oil & gas , metals , mineral resources , foods , pulp & paper and chemicals , among others ..
Length: 44
('value chain upstream', 0.7109)
('marubeni', 0.4173)
('resources foods pulp', 0.2288)
('encompassing wide variety', 0.2081)
('gas metals', 0.1838)


In [ ]:
import yake
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)

for kw in keywords:
	print(kw)


print("SECOND FORM:")
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

NLTK-VADER

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

test_subset=['20170412', 'great', 'bad', 'terrible', 'dog', 'stop', 'good','prices','increasing']


sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in test_subset:
    if (sid.polarity_scores(word)['compound']) >= 0.5:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.5:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)                

print('Positive :',pos_word_list)        
print('Neutral :',neu_word_list)    
print('Negative :',neg_word_list)

Positive : ['HAPPINESS', 'ENJOYS', 'PLEASURE', 'HAPPY', 'IMPRESSING', 'HONORING', 'REWARDING', 'EXCELLING', 'REWARD', 'BRILLIANT']
Neutral : ['COLLABORATIVE', 'INVENTION', 'COLLABORATES', 'SUCCEED', 'IMPROVES', 'BENEFITTED', 'BREAKTHROUGHS', 'LUCRATIVE', 'ACCOMPLISHMENT', 'EXCEPTIONAL', 'ACCOMPLISHES', 'SATISFYING', 'ACCLAIMED', 'EXCITING', 'SURPASSES', 'ENTHUSIASM', 'ADVANCES', 'ENCOURAGES', 'CREATIVELY', 'ENHANCES', 'OUTPERFORMING', 'ATTAINS', 'BENEFIT', 'SATISFIES', 'BOLSTERED', 'SOLVING', 'BOOM', 'ASSURING', 'INNOVATIVENESS', 'INTEGRITY', 'ATTAINMENT', 'ENHANCEMENT', 'FAVORITES', 'GAINED', 'COLLABORATED', 'STABLE', 'STABILIZING', 'LEADING', 'GREATNESS']
Negative : []


In [ ]:
from textblob import TextBlob

def word_polarity(test_subset):
    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]

    for word in test_subset:               
        testimonial = TextBlob(word)
        if testimonial.sentiment.polarity >= 0.5:
            pos_word_list.append(word)
        elif testimonial.sentiment.polarity <= -0.5:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)

    print('Positive :',pos_word_list)        
    print('Neutral :',neu_word_list)    
    print('Negative :',neg_word_list)      

word_polarity(['20170412', 'great', 'bad', 'terrible', 'prices', 'increasing', 'good','not bad','fat'])

Positive : ['great', 'good']
Neutral : ['20170412', 'prices', 'increasing', 'not bad', 'fat']
Negative : ['bad', 'terrible']
